In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time
import os

In [59]:
page_size = 50
start_page = 100
end_page = 120

In [60]:
endpoint = 'https://www.dailymail.co.uk/home/search.html'
params = {
    'offset': '0',
    'size': str(page_size),
    'sel': 'site',
    'searchPhrase': '',
    'sort': 'recent',
    'channel': 'tvshowbiz',
    'type': 'article',
    'days': 'all'
}

In [61]:
page_content = []
for page in range(start_page, end_page):
    params['offset'] = str(page * page_size)
    new_r = requests.get(endpoint, params=params)
    page_content.append(new_r.text)
    time.sleep(1)

In [62]:
end_page * page_size

6000

In [63]:
page_range = str(start_page * page_size) + '_' +str(end_page * page_size)

In [64]:
page_range

'5000_6000'

In [65]:
with open('saved_pages/%s.json' % page_range, 'w') as outfile:
    json.dump(page_content, outfile)

In [66]:
# page_content = []
# for file in files:
#     with open('saved_pages/%s' % file, 'r') as infile:
#         d = json.load(infile)
#         page_content = page_content + d

In [67]:
article_bs = []
for html in page_content:
    bs = BeautifulSoup(html)
    div_sch_result = bs.find_all('div', class_='sch-result')
    article_bs = article_bs + div_sch_result

In [68]:
headline_list = []
for bs_obj in article_bs:
    h3 = bs_obj.find('h3', class_='sch-res-title')
    h4 = bs_obj.find('h4')
    a = h4.find('a').text if h4.find('a') != None else h4.text
    split = h4.text.split(' -')
    date = split[1] if len(split) > 1 else h4.text

    obj = {
        'headline_text': h3.text,
        'article_url': h3.find('a')['href'],
        'article_preview': bs_obj.find('p', class_='sch-res-preview').text,
        'date': date,
        'author': a
    }
    headline_list.append(obj)

In [69]:
df = pd.DataFrame(headline_list)
df = df.drop_duplicates()
df['date'] = df['date'].str.strip()

In [70]:
def replace(old, new):
    return df['date'].str.replace(old, new)

In [71]:
for i in [['\n', ''], ['th', ''], ['nd',''], ['rd',''], ['st', ''],['Augu ', 'August ']]:
    df['date'] = replace(i[0], i[1])

In [72]:
df['date'] = pd.to_datetime(df['date'], format='%B %d %Y, %I:%M:%S %p')
df = df.sort_values('date')

In [73]:
start = str(df['date'].min())
end = str(df['date'].max())

In [74]:
start, end

('2021-12-25 21:25:01', '2021-12-31 12:10:16')

In [75]:
df.to_csv('saved_headlines/%s to %s.csv' % (start, end), index=False)